In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [24]:
import keras
from keras.models import Model, Input, Sequential
from keras.layers import Dense, Dropout, Activation, Flatten

In [4]:
from modis_utils.misc import restore_data, cache_data

In [6]:
plt.rcParams['figure.figsize'] = (20,8)

train_years = list(range(2003,2016))
test_years = list(range(2016,2018))

data_dir = '../groundtruth_mask_lake/ALL/0'
modis_product = 'ALL'
day_period = 8 if modis_product == 'ALL' else 16

In [11]:
percentile = restore_data('percentile_0.dat')
permanent_water_pixels = np.where(percentile > 0.9, 1, 0)

In [7]:
train_list_filenames = [os.path.join(data_dir, str(year), '{}{:03}'.format(year, day), 'masked.dat')
                        for year in train_years for day in range(1,366, day_period)]

test_list_filenames = [os.path.join(data_dir, str(year), '{}{:03}'.format(year, day), 'masked.dat')
                       for year in test_years for day in range(1,366, day_period)]

In [8]:
data = restore_data(os.path.join('cache', 'boundary_vectors_ALL_1.dat'))
data_train = data[0]
data_test = data[1]
data_train.shape, data_test.shape

((598, 1484, 2), (92, 1484, 2))

In [10]:
data_train_X = data_train
data_test_X = data_test

In [48]:
data_train_Y = []
for filename in train_list_filenames:
    img = restore_data(filename)
    img = np.bitwise_or(img, permanent_water_pixels)
    data_train_Y.append(np.sum(img))
data_train_Y = np.asarray(data_train_Y)

data_test_Y = []
for filename in test_list_filenames:
    img = restore_data(filename)
    img = np.bitwise_or(img, permanent_water_pixels)
    data_test_Y.append(np.sum(img))
data_test_Y = np.asarray(data_test_Y)

In [49]:
data_train_Y.shape, data_test_Y.shape

((598,), (92,))

In [29]:
data_train_X.shape[1]

1484

In [33]:
inputs = Input(shape=(data_train_X.shape[1]*data_train_X.shape[2],))
x = Dense(16)(inputs)
x = Dropout(0.5)(x)
x = Activation('tanh')(x)
x = Dense(1)(x)
model = Model(inputs=inputs, outputs=x)

In [34]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 2968)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 16)                47504     
_________________________________________________________________
dropout_7 (Dropout)          (None, 16)                0         
_________________________________________________________________
activation_7 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 17        
Total params: 47,521
Trainable params: 47,521
Non-trainable params: 0
_________________________________________________________________


In [39]:
model.compile(loss='mse', optimizer='adam')

In [50]:
data_train_X = data_train.reshape(data_train.shape[0], -1)
data_test_X = data_test.reshape(data_test.shape[0], -1)
data_train_Y = data_train_Y.reshape(-1, 1)
data_test_Y = data_test_Y.reshape(-1, 1)

In [51]:
data_train_X.shape, data_train_Y.shape, data_test_X.shape, data_test_Y.shape

((598, 2968), (598, 1), (92, 2968), (92, 1))

In [52]:
model.fit(x=data_train_X, y=data_train_Y, batch_size=1, epochs=10, validation_data=(data_test_X, data_test_Y))

Train on 598 samples, validate on 92 samples
Epoch 1/10
598/598 [==============================] - 11s 18ms/step - loss: 2224132628.1204 - val_loss: 2047463151.3043
Epoch 2/10
598/598 [==============================] - 1s 2ms/step - loss: 2223743716.8161 - val_loss: 2046792144.6957
Epoch 3/10
598/598 [==============================] - 1s 2ms/step - loss: 2223382642.5151 - val_loss: 2046133591.6522
Epoch 4/10
598/598 [==============================] - 1s 2ms/step - loss: 2223031680.4281 - val_loss: 2045480793.0435
Epoch 5/10
598/598 [==============================] - 1s 2ms/step - loss: 2222652447.4649 - val_loss: 2044823501.9130
Epoch 6/10
598/598 [==============================] - 1s 2ms/step - loss: 2222300941.0569 - val_loss: 2044167394.7826
Epoch 7/10
598/598 [==============================] - 1s 2ms/step - loss: 2221898239.5719 - val_loss: 2043504325.5652
Epoch 8/10
598/598 [==============================] - 1s 2ms/step - loss: 2221552520.5619 - val_loss: 2042848428.5217
Epoch 9/1

In [53]:
from sklearn.preprocessing import PolynomialFeatures

In [54]:
X = np.arange(6).reshape(3, 2)

In [55]:
poly = PolynomialFeatures(2)

In [57]:
poly.fit_transform(X)

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

In [58]:
poly = PolynomialFeatures(interaction_only=True)